In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
INIT_LR=1e-4 #Initial Learning rate
EPOCHS=20  #Epochs
BS=32  #Batch size

In [3]:
Dataset = r"C:\Users\GVS-PC\Downloads\facemask\Dataset"
CATEGORIES = ["with_mask", "without_mask"]

In [4]:
data = []
labels = []

for category in CATEGORIES:
    path=os.path.join(Dataset, category)
    for img in os.listdir(path):
        img_path=os.path.join(path, img)
        image=load_img(img_path, target_size=(224, 224))
        image=img_to_array(image)
        image=preprocess_input(image)

        data.append(image)
        labels.append(category)

In [5]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

data=np.array(data, dtype="float32")
labels=np.array(labels)

(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

In [6]:
#Augmentation
augmentation=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")

In [7]:
#Base model using MobileNet instead of convolutional layer
baseModel=MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [8]:
#Basic Neural Network with MaxPooling layers 
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7, 7))(headModel)
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(128, activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2, activation="softmax")(headModel)

In [9]:
#The final model
model=Model(inputs=baseModel.input, outputs=headModel)

In [10]:
for layer in baseModel.layers:
    layer.trainable = False
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

[INFO] compiling model...


In [11]:
print("[INFO] training model...")
H = model.fit(augmentation.flow(X_train, Y_train, batch_size=BS),steps_per_epoch=len(X_train) // BS,validation_data=(X_test, Y_test),validation_steps=len(X_test) // BS,epochs=EPOCHS)

[INFO] training model...
Epoch 1/20
2/2 [==============================] - 2s 906ms/step - loss: 0.9832 - accuracy: 0.5682
Epoch 2/20
2/2 [==============================] - 1s 711ms/step - loss: 0.5928 - accuracy: 0.7344
Epoch 3/20
2/2 [==============================] - 1s 723ms/step - loss: 0.6726 - accuracy: 0.6406
Epoch 4/20
2/2 [==============================] - 2s 922ms/step - loss: 0.5008 - accuracy: 0.7727
Epoch 5/20
2/2 [==============================] - 1s 298ms/step - loss: 0.5244 - accuracy: 0.7045
Epoch 6/20
2/2 [==============================] - 2s 771ms/step - loss: 0.3998 - accuracy: 0.8438
Epoch 7/20
2/2 [==============================] - 2s 883ms/step - loss: 0.4025 - accuracy: 0.8125
Epoch 8/20
2/2 [==============================] - 2s 914ms/step - loss: 0.3261 - accuracy: 0.8594
Epoch 9/20
2/2 [==============================] - 2s 1s/step - loss: 0.2598 - accuracy: 0.8864
Epoch 10/20
2/2 [==============================] - 2s 1s/step - loss: 0.3051 - accuracy: 0.8636


In [12]:
print("[INFO] evaluating network...")
predIdxs = model.predict(X_test, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)

[INFO] evaluating network...


In [13]:
print("[INFO] saving mask detector model...")
model.save("mask_detector.model", save_format="h5")

[INFO] saving mask detector model...
